code step by step:

1. **Import Libraries**: In the first part, we import all the necessary libraries and modules that we'll be using throughout the code. These include libraries for data manipulation (`numpy`, `pandas`, `polars`), machine learning models (`lightgbm`, `catboost`), file handling (`joblib`, `Path`), and other utilities (`gc`, `glob`).

2. **Define Classes for Data Processing**: The code defines two classes: `Pipeline` and `Aggregator`. These classes encapsulate methods for data preprocessing and feature engineering, respectively.

3. **Define Functions for Data Reading and Feature Engineering**: Several functions are defined to read data files, perform feature engineering, and convert data to a more memory-efficient format. These functions include `read_file`, `read_files`, `feature_eng`, `to_pandas`, and `reduce_mem_usage`.

4. **Load Trained Models and Model Metadata**: The code loads trained models (`lgb_models`, `cat_models`) and their associated metadata (`lgb_notebook_info`, `cat_notebook_info`) from disk. These models are later used for making predictions on the train data.

5. **Define train Data Paths and Load train Data**: train data paths are defined, and the train data is loaded using the previously defined functions for reading data files. The loaded train data is stored in a dictionary called `data_store`.

6. **Perform Feature Engineering on train Data**: The loaded train data is passed through the feature engineering pipeline (`feature_eng`) to generate features required for making predictions.

7. **Generate Predictions**: The `VotingModel` class is used to generate predictions on the train data. This class averages the predictions from multiple individual models to obtain the final prediction probabilities.

8. **Save Predictions to Submission File**: The predicted probabilities are saved to a CSV file (`submission.csv`) in the format required for submission. The submission file is based on a sample submission file provided earlier (`sample_submission.csv`).

9. **Display Submission DataFrame**: Finally, the submission DataFrame (`df_subm`) is displayed, showing the case IDs and corresponding predicted scores.



In [15]:
import joblib  # Import joblib for saving and loading models
from pathlib import Path  # Import Path for working with file paths
import gc  # Import gc for garbage collection
from glob import glob  # Import glob for file matching
import numpy as np  # Import numpy for numerical computing
import pandas as pd  # Import pandas for data manipulation
import polars as pl  # Import polars for fast data manipulation
from sklearn.base import BaseEstimator, RegressorMixin  # Import BaseEstimator and RegressorMixin from sklearn.base
from sklearn.metrics import roc_auc_score  # Import roc_auc_score from sklearn.metrics
import lightgbm as lgb  # Import lightgbm for gradient boosting

import warnings  # Import warnings to ignore warnings
warnings.filterwarnings('ignore')  # Ignore warnings

USER = ''
ROOT = Path(f"/home/{USER}/public/home-credit-credit-risk-model-stability")  # Define ROOT path

In [16]:
class Pipeline:
    # Method to set data types for specific columns in a DataFrame
    def set_table_dtypes(df):
        for col in df.columns:
            if col in ["case_id", "WEEK_NUM", "num_group1", "num_group2"]:
                df = df.with_columns(pl.col(col).cast(pl.Int64))
            elif col in ["date_decision"]:
                df = df.with_columns(pl.col(col).cast(pl.Date))
            elif col[-1] in ("P", "A"):
                df = df.with_columns(pl.col(col).cast(pl.Float64))
            elif col[-1] in ("M",):
                df = df.with_columns(pl.col(col).cast(pl.String))
            elif col[-1] in ("D",):
                df = df.with_columns(pl.col(col).cast(pl.Date))
        return df

    # Method to handle date columns and calculate time differences
    def handle_dates(df):
        for col in df.columns:
            if col[-1] in ("D",):
                df = df.with_columns(pl.col(col) - pl.col("date_decision"))  # Calculate time differences
                df = df.with_columns(pl.col(col).dt.total_days())  # Convert time differences to total days
        df = df.drop("date_decision", "MONTH")  # Drop unnecessary columns
        return df

    # Method to filter out columns based on missing values and frequency
    def filter_cols(df):
        for col in df.columns:
            if col not in ["target", "case_id", "WEEK_NUM"]:
                isnull = df[col].is_null().mean()
                if isnull > 0.7:
                    df = df.drop(col)  # Drop columns with more than 70% missing values
        
        for col in df.columns:
            if (col not in ["target", "case_id", "WEEK_NUM"]) & (df[col].dtype == pl.String):
                freq = df[col].n_unique()
                if (freq == 1) | (freq > 200):
                    df = df.drop(col)  # Drop columns with only one unique value or more than 200 unique values
        
        return df

In [17]:
class Aggregator:
    # Method to aggregate numerical features
    def num_expr(df):
        cols = [col for col in df.columns if col[-1] in ("P", "A")]  # Select numerical columns
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]  # Calculate max
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]  # Calculate last
        expr_mean = [pl.mean(col).alias(f"mean_{col}") for col in cols]  # Calculate mean
        expr_median = [pl.median(col).alias(f"median_{col}") for col in cols]  # Calculate median
        expr_var = [pl.var(col).alias(f"var_{col}") for col in cols]  # Calculate variance
        return expr_max + expr_last + expr_mean 

    # Method to aggregate date features
    def date_expr(df):
        cols = [col for col in df.columns if col[-1] in ("D")]  # Select date columns
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]  # Calculate max
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]  # Calculate last
        expr_mean = [pl.mean(col).alias(f"mean_{col}") for col in cols]  # Calculate mean
        expr_median = [pl.median(col).alias(f"median_{col}") for col in cols]  # Calculate median
        return expr_max + expr_last + expr_mean 

    # Method to aggregate string features
    def str_expr(df):
        cols = [col for col in df.columns if col[-1] in ("M",)]  # Select string columns
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]  # Calculate max
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]  # Calculate last
        return expr_max + expr_last

    # Method to aggregate other features
    def other_expr(df):
        cols = [col for col in df.columns if col[-1] in ("T", "L")]  # Select other columns
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]  # Calculate max
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]  # Calculate last
        return expr_max + expr_last

    # Method to aggregate count features
    def count_expr(df):
        cols = [col for col in df.columns if "num_group" in col]  # Select count columns
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]  # Calculate max
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]  # Calculate last
        return expr_max + expr_last

    # Method to get all aggregation expressions
    def get_exprs(df):
        exprs = Aggregator.num_expr(df) + \
                Aggregator.date_expr(df) + \
                Aggregator.str_expr(df) + \
                Aggregator.other_expr(df) + \
                Aggregator.count_expr(df)
        return exprs

In [18]:
def read_file(path, depth=None):
    # Read parquet file into a Polars DataFrame
    df = pl.read_parquet(path)
    # Set table data types using Pipeline method
    df = df.pipe(Pipeline.set_table_dtypes)
    # Aggregate features if depth is specified
    if depth in [1, 2]:
        df = df.group_by("case_id").agg(Aggregator.get_exprs(df)) 
    return df


In [19]:
def read_files(regex_path, depth=None):
    chunks = []
    # Iterate over files matching the regex pattern
    for path in glob(str(regex_path)):
        # Read parquet file into a Polars DataFrame
        df = pl.read_parquet(path)
        # Set table data types using Pipeline method
        df = df.pipe(Pipeline.set_table_dtypes)
        # Aggregate features if depth is specified
        if depth in [1, 2]:
            df = df.group_by("case_id").agg(Aggregator.get_exprs(df))
        chunks.append(df)
    # Concatenate DataFrames and drop duplicate rows based on "case_id"
    df = pl.concat(chunks, how="vertical_relaxed").unique(subset=["case_id"])
    return df

In [20]:
def feature_eng(df_base, depth_0, depth_1, depth_2):
    # Add month and weekday features based on "date_decision"
    df_base = df_base.with_columns(
        month_decision = pl.col("date_decision").dt.month(),
        weekday_decision = pl.col("date_decision").dt.weekday(),
    )
    # Join additional depth DataFrames
    for i, df in enumerate(depth_0 + depth_1 + depth_2):
        df_base = df_base.join(df, how="left", on="case_id", suffix=f"_{i}")
    # Handle dates using Pipeline method
    df_base = df_base.pipe(Pipeline.handle_dates)
    return df_base

In [21]:
def to_pandas(df_data, cat_cols=None):
    # Convert Polars DataFrame to pandas DataFrame
    df_data = df_data.to_pandas()
    # Convert categorical columns to category data type
    if cat_cols is None:
        cat_cols = list(df_data.select_dtypes("object").columns)
    df_data[cat_cols] = df_data[cat_cols].astype("category")
    return df_data, cat_cols

In [22]:
def reduce_mem_usage(df):
    """ 
    Iterate through all the columns of a dataframe and modify the data type
    to reduce memory usage.
    """
    start_mem = df.memory_usage().sum() / 1024**2  # Memory usage before optimization
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        if str(col_type)=="category":
            continue
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            continue
    end_mem = df.memory_usage().sum() / 1024**2  # Memory usage after optimization
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [9]:
# lgb_notebook_info = joblib.load('/kaggle/input/homecredit-models-public/other/lgb/1/notebook_info.joblib')

# # Print notebook information
# print(f"- [lgb] notebook_start_time: {lgb_notebook_info['notebook_start_time']}")
# print(f"- [lgb] description: {lgb_notebook_info['description']}")

# # Load columns and categorical columns
# cols = lgb_notebook_info['cols']
# cat_cols = lgb_notebook_info['cat_cols']
# print(f"- [lgb] len(cols): {len(cols)}")
# print(f"- [lgb] len(cat_cols): {len(cat_cols)}")

# # Load LightGBM models
# lgb_models = joblib.load('/kaggle/input/homecredit-models-public/other/lgb/1/lgb_models.joblib')
# lgb_models

In [10]:
# # Load categorical model notebook information
# cat_notebook_info = joblib.load('/kaggle/input/homecredit-models-public/other/cat/1/notebook_info.joblib')

# # Print notebook information
# print(f"- [cat] notebook_start_time: {cat_notebook_info['notebook_start_time']}")
# print(f"- [cat] description: {cat_notebook_info['description']}")

# # Load categorical models
# cat_models = joblib.load('/kaggle/input/homecredit-models-public/other/cat/1/cat_models.joblib')
# cat_models

In [36]:
# Define the directory path for the train data
train_DIR = ROOT / "parquet_files/train"

# Create a dictionary to store different dataframes generated from reading parquet files
data_store = {
    # Read the base train data and store it with the key 'df_base'
    "df_base": read_file(train_DIR / "train_base.parquet"),
    
    # Read depth 0 data, which includes static data and additional files matching a pattern
    "depth_0": [
        read_file(train_DIR / "train_static_cb_0.parquet"),
        read_files(train_DIR / "train_static_0_*.parquet"),
    ],
    
    # Read depth 1 data, including various files related to applicant previous applications, tax registries,
    # credit bureau data, and other information
    "depth_1": [
        read_files(train_DIR / "train_applprev_1_*.parquet", 1),
        read_file(train_DIR / "train_tax_registry_a_1.parquet", 1),
        read_file(train_DIR / "train_tax_registry_b_1.parquet", 1),
        read_file(train_DIR / "train_tax_registry_c_1.parquet", 1),
        read_files(train_DIR / "train_credit_bureau_a_1_*.parquet", 1),
        read_file(train_DIR / "train_credit_bureau_b_1.parquet", 1),
        read_file(train_DIR / "train_other_1.parquet", 1),
        read_file(train_DIR / "train_person_1.parquet", 1),
        read_file(train_DIR / "train_deposit_1.parquet", 1),
        read_file(train_DIR / "train_debitcard_1.parquet", 1),
    ],
    
    # Read depth 2 data, which includes additional credit bureau data, applicant previous applications,
    # and personal information
    "depth_2": [
        read_file(train_DIR / "train_credit_bureau_b_2.parquet", 2),
        read_files(train_DIR / "train_credit_bureau_a_2_*.parquet", 2),
        read_file(train_DIR / "train_applprev_2.parquet", 2),
        read_file(train_DIR / "train_person_2.parquet", 2)
    ]
}

In [ ]:
# # Perform feature engineering on the train data using the provided data store
df_train = feature_eng(**data_store)

# # Print the shape of the train data before further processing
print("train data shape:\t", df_train.shape)

# # Clean up memory by deleting the data store and running garbage collection
del data_store
gc.collect()

# # Select columns of interest from the train data
# df_train = df_train.select(['case_id'] + cols)

# # Convert the train data to a pandas DataFrame and optimize memory usage
# df_train, cat_cols = to_pandas(df_train, cat_cols)
# df_train = reduce_mem_usage(df_train)

# # Set the case_id column as the index of the DataFrame
# df_train = df_train.set_index('case_id')

# # Print the shape of the train data after processing
# print("train data shape:\t", df_train.shape)

# # Run garbage collection to clean up memory
# gc.collect()

In [ ]:
# class VotingModel(BaseEstimator, RegressorMixin):
#     def __init__(self, estimators):
#         super().__init__()
#         self.estimators = estimators
        
#     def fit(self, X, y=None):
#         """
#         Fit the VotingModel.
        
#         Parameters:
#         - X: array-like or sparse matrix of shape (n_samples, n_features)
#             The input samples.
#         - y: array-like of shape (n_samples,), default=None
#             The target values.
            
#         Returns:
#         - self: object
#             Returns self.
#         """
#         return self
    
#     def predict(self, X):
#         """
#         Predict regression target for X.
        
#         Parameters:
#         - X: array-like or sparse matrix of shape (n_samples, n_features)
#             The input samples.
            
#         Returns:
#         - y_preds: array-like of shape (n_samples,)
#             The predicted target values.
#         """
#         y_preds = [estimator.predict(X) for estimator in self.estimators]
#         return np.mean(y_preds, axis=0)
     
#     def predict_proba(self, X):      
#         """
#         Predict class probabilities for X.
        
#         Parameters:
#         - X: array-like or sparse matrix of shape (n_samples, n_features)
#             The input samples.
            
#         Returns:
#         - proba: array-like of shape (n_samples, n_classes)
#             Class probabilities of the input samples.
#         """
#         # lgb
#         y_preds = [estimator.predict_proba(X) for estimator in self.estimators[:5]]
        
#         # cat        
#         X[cat_cols] = X[cat_cols].astype(str)
#         y_preds += [estimator.predict_proba(X) for estimator in self.estimators[-5:]]
        
#         return np.mean(y_preds, axis=0)

In [ ]:
model = VotingModel(lgb_models + cat_models)
len(model.estimators)

In [ ]:
# Predict probabilities for the train data
y_pred = pd.Series(model.predict_proba(df_train)[:, 1], index=df_train.index)

# Read the sample submission file
df_subm = pd.read_csv(ROOT / "sample_submission.csv")
df_subm = df_subm.set_index("case_id")

# Assign predicted probabilities to the submission dataframe
df_subm["score"] = y_pred

# Save the submission dataframe to a CSV file
df_subm.to_csv("submission.csv")

# Display the submission dataframe
df_subm